# First steps

This tutorial demonstrates the basic usage of *Spindles.jl* to create spindles
and query basic properties.

## Creating a spindle

In this tutorial, we will be working with one of the simplest examples of a spindle: a cube.
For example, the unit cube in 3D is given by all points $x = (x_1,x_2,x_3)$ that satisfy
$$
\begin{aligned}
-1 \le x_1 &\le 1 \\
-1 \le x_2 &\le 1 \\
-1 \le x_3 &\le 1
\end{aligned}
$$

In matrix notation, this is equivalent to the system $Ax \le b$ where

In [1]:
A = [1 0 0; -1 0 0; 0 1 0; 0 -1 0; 0 0 1; 0 0 -1]

6×3 Matrix{Int64}:
  1   0   0
 -1   0   0
  0   1   0
  0  -1   0
  0   0   1
  0   0  -1

and

In [2]:
b = [1, 1, 1, 1, 1, 1]

6-element Vector{Int64}:
 1
 1
 1
 1
 1
 1

*Spindles.jl* provides a data type for representing and analyzing polytopes:
`Polytope`. We may create an object of this type from our data `A` and `b` as follows:

In [3]:
using Spindles # hide
cube = Polytope(A, b)

Polytope{Rational{BigInt}} in 3-space

> **Tip**
>
> The 3D unit cube can also be created using the generator function `cube` implemented
> in *Spindles.jl*.

What sets a spindle apart from a general polytope is the existence of two vertices
(the apices) whose incident facets partition the set of all facets.
We may check `cube` for the existence of such a pair of vertices by running

In [4]:
apices(cube)

2-element Vector{Int64}:
 1
 8

This returns the two indices of the apices as they appear in the list of all (eight) vertices
of `cube`. To list the vertices explicitly, do

In [5]:
collect(vertices(cube))

8-element Vector{Vector{Rational{BigInt}}}:
 [-1, -1, -1]
 [1, -1, -1]
 [-1, 1, -1]
 [1, 1, -1]
 [-1, -1, 1]
 [1, -1, 1]
 [-1, 1, 1]
 [1, 1, 1]

> **Note**
>
> By default, *Spindles.jl* uses exact rational arithmetic. Note that the components of each
> vertex returned by `vertices` are of type `Rational` with numerators
> and denominators of type `BigInt` to avoid integer overflows (see the Julia documentation pages on
> [rational numbers](https://docs.julialang.org/en/v1/manual/complex-and-rational-numbers/#Rational-Numbers) and
> [arbitrary-precision arithmetic](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Arbitrary-Precision-Arithmetic)).

So the first and last vertex in the list above can take the role of the apices of `cube`.
However, these two are not unique. In fact, for a cube there are many possible pairs of apices:
Take an arbitrary vertex and its antipodal one, i.e., the vertex obtained by flipping the sign
of each component. To prescribe an apex, pass its index as an additional argument
to the function `apices`, which then tries to find a matching second apex:

In [6]:
apx = apices(cube, 3)

2-element Vector{Int64}:
 3
 6

## Working with the graph

We may even compute the distance between those two apices in the graph of `cube`:

In [7]:
dist(cube, apx...)

3

Behind the scenes, the call to `dist` first computes the graph of `cube`.
The graph can also be accessed directly using `graph`, which returns a graph
of a type defined by the [Graphs.jl](https://github.com/JuliaGraphs/Graphs.jl) package.
For instance, we may verify the well-known fact that cubes are simple by using the
functions `dim` and [`Graphs.degree`](https://juliagraphs.org/Graphs.jl/stable/core_functions/core/#Graphs.degree):

In [8]:
using Graphs: degree
all(degree(graph(cube)) .== dim(cube))

true

To make verifying this property more convenient, *Spindles.jl* provides the function
`issimple` that checks a polytope for being simple:

In [9]:
issimple(cube)

true

## Counting and enumerating faces

*Spindles.jl* also provides functions to count and enumerate the faces of `cube`. The following call to
`facesofdim` returns all two-dimensional faces, each one given by the indices of its
incident facets.

> **Note**
>
> Note here that facet indices refer to the corresponding rows of the coefficient matrix `A`.

In [10]:
collect(facesofdim(cube, 2))

6-element Vector{Vector{Int64}}:
 [6]
 [4]
 [2]
 [1]
 [3]
 [5]

Given that the two-dimensional faces of a cube are precisely its six facets,
this should not be too surprising.

To count the faces of a given dimension without explicitly producing a list,
use the function `nfacesofdim`. For example, we may compute
the [f-vector](https://en.wikipedia.org/wiki/Polyhedral_combinatorics#Faces_and_face-counting_vectors)
of `cube` as follows.

In [11]:
nfacesofdim.(cube, 0:(dim(cube)-1))

3-element Vector{Int64}:
  8
 12
  6

This tells us that `cube` has 8 vertices, 12 edges, and 6 facets.

Next, let's list all vertices that are incident to the first facet.

In [12]:
for v in incidentvertices(cube, [1])
    println(collect(vertices(cube))[v])
end

Rational{BigInt}[1, -1, -1]
Rational{BigInt}[1, 1, -1]
Rational{BigInt}[1, -1, 1]
Rational{BigInt}[1, 1, 1]


As expected, we obtain precisely the four vertices whose first component is equal to one.

> **Note**
>
> `vertices`, like `facesofdim`, returns an iterator.
> To access a specific element,
> use [`collect`](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Any})
> as in the code above.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*